In [1]:
'''데이터 생성 스크립트'''
# bboxes = data['bboxes'] # nested array: [[x, y, w, h],...]
# scores = data['scores'] # [0.6, 0.89,...]
# labels = data['labels'] # [0, 4,...] # yolo 라벨 0-23
# timestamp = data['timestamp'] # unix
# width = data['width'] # 이미지 width
# height = data['height'] # 이미지 height
# region_and_name = data['region_and_name'] # 카메라 정보. id?
# img_directory = data['img_directory'] # 이미지 저장 경로
# dict_values([[[580.052604675293, 217.65965223312378, 60.063819885253906, 58.98589611053467], [330.85792541503906, 175.94241857528687, 33.85063171386719, 39.26307678222656]], [0.6402683258056641, 0.5894748568534851], [0, 4], 1693530106898, 640, 360, '지역', 'C:/cctv_images/지역_1693530106898.jpg'])

'데이터 생성 스크립트'

In [2]:
import pandas as pd

columns = ['cctv_id', 'cctv_name', 'center_name', 'timestamp', 'x', 'y', 'bwidth', 'bheight', 'width', 'height', 'score', 'label', 'img_directory']

df = pd.DataFrame(columns=columns)
df

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory


In [3]:
cctv_df = pd.read_csv(r'static\CCTV.csv')
cctv_df.head()

,Unnamed: 0,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,URL
0,0,1,L280034,가덕영업소,거제교통정보센터,128.817344,35.025620,https://www.utic.go.kr/view/map/openDataCctvSt...
1,1,2,L280005,거가1교,거제교통정보센터,128.731926,35.017735,https://www.utic.go.kr/view/map/openDataCctvSt...
2,2,3,L280003,거가1교(저도방향),거제교통정보센터,128.721416,35.017084,https://www.utic.go.kr/view/map/openDataCctvSt...
3,3,4,L280054,거가2교,거제교통정보센터,128.758293,35.014669,https://www.utic.go.kr/view/map/openDataCctvSt...
4,4,5,L280001,거제면 귀목정,거제교통정보센터,128.581956,34.862783,https://www.utic.go.kr/view/map/openDataCctvSt...


In [4]:
cctv_df.shape

(7746, 8)

In [6]:
import random
import time
import os
class GenerateRandomData:
    def __init__(self, x1=None, y1=None, score=None, label=None, width=None, height=None, cctvid=None, img_directory_pre=None):
        '입력 안 하면 랜덤으로 x1, y1, label 설정됨'
        self._width = width
        self._height = height
        self._generate_width_height()
        self.x1 = x1
        self.y1 = y1
        if not all([self.x1, self.y1]):
            self.x1, self.y1 = self.generate_starting_point()
        self.bbox = self.generate_bbox(self.x1, self.y1)
        self.score = score or self.generate_score()
        self.label = label or self.generate_label()
        self._cctvid = cctvid or self._generate_region_and_name()[0]
        self._img_directory_pre = img_directory_pre or self._generate_img_directory_pre()
        self.img_directory = os.path.join(self._img_directory_pre, self._cctvid + '_' + str(int(time.time()*1000)) + '.jpg')

    def generate_starting_point(self):
        x1 = random.uniform(0, self._width)
        y1 = random.uniform(0, self._height)
        return x1, y1

    def generate_bbox(self, x, y):
        '''
        x,y coordinates로 bbox 만들기\n
        x, y, bwidth, bheight로 구성
        '''
        bwidth = random.uniform(10,100)
        bheight = random.uniform(10,100)
        # 이미지 사이즈 안 넘어가게
        if x + bwidth > self._width: bwidth = self._width - x
        if y + bheight > self._height: bheight = self._height - y
        return [x, y, bwidth, bheight]
    
    def generate_score(self):
        'score 생성'
        return random.uniform(0.5, 1)
    
    def generate_label(self):
        'label 생성'
        return random.randrange(0, 23)
    
    def _generate_width_height(self, new=False):
        'width, height 생성'
        self._width = random.choice([640, 1280, 1600, 1920]) if new else (self._width if self._width else random.choice([640, 1280, 1600, 1920]))
        self._height = int(self._width * (9/16)) if new else (self._height if self._height else int(self._width * (9/16)))
        return self._width, self._height
    
    def generate_timestamp(self):
        'timestamp 생성'
        return int(time.time()*1000)

    def _generate_region_and_name(self, new=False):
        'region 정보 생성'
        idx = random.randrange(0, 7746)
        cctv_id, cctv_name, center_name = cctv_df.iloc[idx, 2:5]
        return [cctv_id, cctv_name, center_name]
    
    def _generate_img_directory_pre(self, new=False):
        '파일 부모 저장경로 생성'
        return random.choice([r'C:/cctv_images', r'/home/cctv_images', r'~/Downloads/cctv_images'])
    
    def generate_img_directory(self, timestamp):
        '파일 저장경로 생성'
        return os.path.join(self._img_directory_pre, self._cctvid + '_' + str(timestamp) + '.jpg')
    
    def generate_bulk_data(self, count=100):
        'bulk data 생성'
        dict_ = {column: [] for column in columns}
        # width, height 생성
        width, height = self._generate_width_height(new=True)
        # cctv 정보 생성
        cctv_id, cctv_name, center_name = self._generate_region_and_name(new=True)
        # img_directory_pre 생성
        img_directory_pre = self._generate_img_directory_pre(new=True)
        for _ in range(count):
            # columns = ['bboxes', 'scores', 'labels', 'timestamp', 'width', 'height', 'cctvid', 'region_and_name', 'centername', 'img_directory']
            # x1,y1 생성
            x, y = self.generate_starting_point()
            # bbox 생성
            bbox = self.generate_bbox(x, y)
            # score 생성
            score = self.generate_score()
            # label 생성
            label = self.generate_label()
            # timestamp 생성
            timestamp = self.generate_timestamp()
            # img_directory 생성
            img_directory = os.path.join(img_directory_pre, cctv_id + '_' + str(timestamp) + '.jpg')
            dict_['cctv_id'].append(cctv_id)
            dict_['cctv_name'].append(cctv_name)
            dict_['center_name'].append(center_name)
            dict_['timestamp'].append(timestamp)
            dict_['x'].append(bbox[0])
            dict_['y'].append(bbox[1])
            dict_['bwidth'].append(bbox[2])
            dict_['bheight'].append(bbox[3])
            dict_['width'].append(width)
            dict_['height'].append(height)
            dict_['score'].append(score)
            dict_['label'].append(label)
            dict_['img_directory'].append(img_directory)
        return pd.DataFrame.from_dict(dict_)

generator = GenerateRandomData()
generator.generate_label()
generator._generate_region_and_name()
df_ = generator.generate_bulk_data()

In [7]:
df = pd.DataFrame(columns=columns)

for _ in range(1000):
    df = pd.concat([df, generator.generate_bulk_data(100)])
df

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory
0,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,405.120719,345.587006,49.981266,14.412994,640,360,0.513934,22,/home/cctv_images\L360019_1693549732006.jpg
1,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,472.996643,186.767781,11.183475,99.880325,640,360,0.844032,21,/home/cctv_images\L360019_1693549732006.jpg
2,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,320.749935,213.912469,15.889107,54.279654,640,360,0.920935,15,/home/cctv_images\L360019_1693549732006.jpg
3,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732007,258.270515,229.260962,46.407064,26.157999,640,360,0.700021,16,/home/cctv_images\L360019_1693549732007.jpg
4,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732007,455.445365,67.196998,35.941937,89.662423,640,360,0.551828,19,/home/cctv_images\L360019_1693549732007.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,583.545556,353.671830,56.454444,6.328170,640,360,0.579471,9,~/Downloads/cctv_images\E901175_1693549738714.jpg
96,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,446.378427,329.417659,23.418138,30.582341,640,360,0.816135,11,~/Downloads/cctv_images\E901175_1693549738714.jpg
97,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,549.230133,80.633101,30.954198,93.361822,640,360,0.544152,22,~/Downloads/cctv_images\E901175_1693549738714.jpg
98,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,536.783054,0.873910,13.895922,11.724266,640,360,0.885060,2,~/Downloads/cctv_images\E901175_1693549738714.jpg


In [8]:
df.to_csv('generated_data4.csv', index=False)

In [16]:
df_ = pd.read_csv('generated_data4.csv')
df_

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory
0,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,405.120719,345.587006,49.981266,14.412994,640,360,0.513934,22,/home/cctv_images\L360019_1693549732006.jpg
1,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,472.996643,186.767781,11.183475,99.880325,640,360,0.844032,21,/home/cctv_images\L360019_1693549732006.jpg
2,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732006,320.749935,213.912469,15.889107,54.279654,640,360,0.920935,15,/home/cctv_images\L360019_1693549732006.jpg
3,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732007,258.270515,229.260962,46.407064,26.157999,640,360,0.700021,16,/home/cctv_images\L360019_1693549732007.jpg
4,L360019,레이크타운3차 사거리,천안교통정보센터,1693549732007,455.445365,67.196998,35.941937,89.662423,640,360,0.551828,19,/home/cctv_images\L360019_1693549732007.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,583.545556,353.671830,56.454444,6.328170,640,360,0.579471,9,~/Downloads/cctv_images\E901175_1693549738714.jpg
99996,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,446.378427,329.417659,23.418138,30.582341,640,360,0.816135,11,~/Downloads/cctv_images\E901175_1693549738714.jpg
99997,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,549.230133,80.633101,30.954198,93.361822,640,360,0.544152,22,~/Downloads/cctv_images\E901175_1693549738714.jpg
99998,E901175,[국도23호선]익산동련교차로,국가교통정보센터(국도),1693549738714,536.783054,0.873910,13.895922,11.724266,640,360,0.885060,2,~/Downloads/cctv_images\E901175_1693549738714.jpg


In [14]:
a

dict_values([{0: 'E901175', 1: 'E901175', 2: 'E901175', 3: 'E901175', 4: 'E901175', 5: 'E901175', 6: 'E901175', 7: 'E901175', 8: 'E901175', 9: 'E901175', 10: 'E901175', 11: 'E901175', 12: 'E901175', 13: 'E901175', 14: 'E901175', 15: 'E901175', 16: 'E901175', 17: 'E901175', 18: 'E901175', 19: 'E901175', 20: 'E901175', 21: 'E901175', 22: 'E901175', 23: 'E901175', 24: 'E901175', 25: 'E901175', 26: 'E901175', 27: 'E901175', 28: 'E901175', 29: 'E901175', 30: 'E901175', 31: 'E901175', 32: 'E901175', 33: 'E901175', 34: 'E901175', 35: 'E901175', 36: 'E901175', 37: 'E901175', 38: 'E901175', 39: 'E901175', 40: 'E901175', 41: 'E901175', 42: 'E901175', 43: 'E901175', 44: 'E901175', 45: 'E901175', 46: 'E901175', 47: 'E901175', 48: 'E901175', 49: 'E901175', 50: 'E901175', 51: 'E901175', 52: 'E901175', 53: 'E901175', 54: 'E901175', 55: 'E901175', 56: 'E901175', 57: 'E901175', 58: 'E901175', 59: 'E901175', 60: 'E901175', 61: 'E901175', 62: 'E901175', 63: 'E901175', 64: 'E901175', 65: 'E901175', 66: 'E